# Regional 3D plot

Load a regional crop of the earth topography, texture and plot some temperature data


In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import lavavu
import os
import accessvis

## Load some temperature data

In [ ]:
#https://thredds.nci.org.au/thredds/catalog/fs38/publications/CMIP6/CMIP/CSIRO-ARCCSS/ACCESS-CM2/historical/r10i1p1f1/day/hfls/gn/latest/catalog.html?dataset=fs38/CMIP/CSIRO-ARCCSS/ACCESS-CM2/historical/r10i1p1f1/day/hfls/gn/latest/
thredds_server = "https://thredds.nci.org.au/thredds/dodsC/fs38/publications/"
dl_server = "https://thredds.nci.org.au/thredds/fileServer/fs38/publications/"

#Surface temperature
ver = "CMIP6/CMIP/CSIRO-ARCCSS/ACCESS-CM2/1pctCO2"  #historical/"
fpath = "/r1i1p1f1/Amon/tas/gn/latest/tas_Amon_ACCESS-CM2_1pctCO2_r1i1p1f1_gn_095001-109912.nc"
var = 'tas'
url = f"{thredds_server}{ver}{fpath}"

In [ ]:
print(f'{url}.html')

In [ ]:
ds = xr.open_dataset(url)

In [ ]:
ds

In [ ]:
times = ds['time']
data = ds[var]

In [ ]:
data

In [ ]:
data[0].plot()

## Select a region to plot

In [ ]:
#South-eastern Australia (lat,lon) (y,x)
TL = (-29, 135)
BR = (-45, 160)

## Crop the data to selected region

In [ ]:
#Prepare cropped region
min_lat = BR[0]
max_lat = TL[0]
min_lon = TL[1]
max_lon = BR[1]

cropped_ds = ds.sel(lat=slice(min_lat,max_lat), lon=slice(min_lon,max_lon))

In [ ]:
cropped_ds

In [ ]:
cropped_ds['tas'][0].plot()

In [ ]:
cropped_ds.lat

In [ ]:
cropped_ds.lon

In [ ]:
latitude = cropped_ds.lat.values
longitude = cropped_ds.lon.values

## 3D Plot

In [ ]:
#Use higher res topo when cropping to region
accessvis.resolution_selection(default=3)

In [ ]:
#Pass the crop box to plot_region to get regional topography view
V_EXAG = 20.0 #Vertical exaggeration
cropbox=((latitude[0], longitude[0]), (latitude[-1], longitude[-1]))
lv = accessvis.plot_region(cropbox=cropbox, vertical_exaggeration=V_EXAG, bathymetry=False)

In [ ]:
#Setup lighting
lv.set_properties(diffuse=0.6, ambient=0.85, specular=0.25, shininess=0.03, light=[1,1,0.98,1], lightpos=[0,0,10000,1])

In [ ]:
lv.display((640,480))

In [ ]:
#Get first timestep temperature, converted to C
tas = np.array(cropped_ds[var][0])-273.15

In [ ]:
plt.imshow(tas)

In [ ]:
#Apply a colourmap
rgba = accessvis.array_to_rgba(tas, colourmap='coolwarm')

In [ ]:
plt.imshow(rgba)

In [ ]:
topo = lv.objects['surface']

In [ ]:
#Grab the vertices from our existing topo surface
#We use vertices_copy() as we want to modify the data for a new surface,
#vertices() returns a reference to the actual data and without making a copy
#any changes would also modify the topo vertices in-place
V = topo.data.vertices_copy[0]

In [ ]:
#Add a 5 metre offset for the overlay surface
offset = 5 * accessvis.MtoLL * V_EXAG
V[::,::,2] = V[::,::,2] + offset

In [ ]:
#Create a new surface to plot temperature on
tsurf = lv.triangles('temperature', vertices=V, cullface=True, texture="blank.png", lit=False, alpha=0.6)

In [ ]:
lv.reload()

In [ ]:
#Load the colourmapped data
tsurf.texture(rgba)

In [ ]:
cmap = lv.colourmap('coolwarm', range=(0,50))
cb = lv.colourbar()
#cb['range'] = (0,50)

In [ ]:
lv.reset()
lv.translation(0.0, 0.0, -11)
lv.display((800,600))

In [ ]:
# Set a different camera view

In [ ]:
lv.translation(0, 1.5, -11.5)
lv.rotation(-37, 0, 0)

In [ ]:
lv.display((1920//2,1080//2))

## Animation

Plot temperature data for January over full dataset

In [ ]:
from tqdm.notebook import tqdm
step = 12
fn = 'regional_temp_jan.mp4'
with lv.video(filename=fn, fps=60, quality=3, resolution=(1920//2,1080//2), params="autoplay"):    
    for ts in tqdm(range(0,len(times),step)):
        lv["title"] = f"+y{ts//12+1} m{ts%12+1}"
        tas = np.array(cropped_ds[var][ts])-273.15
        rgba = accessvis.array_to_rgba(tas, colourmap='coolwarm')
        tsurf.texture(rgba)
        lv.render()
    #Final frames
    for i in range(40):
        lv.render()